# Coursera Capstone Week 3-1 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# BeautifulSoup reqd to parse HTML from Wikipedia
#import bs4 as bs4
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


##### Get data from  Wikipedia

wikipedia is banned in our country (Turkey). I was populate data from wikizeroo.net

wikizeroo.net not accepts pd.read_html(url).. I was create dataframe with loop.



In [3]:
# rawpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wikizeroourl = 'http://www.wikizeroo.net/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N'

rawpage = requests.get(wikizeroourl)
soup = BeautifulSoup(rawpage.text, 'html.parser')
table = soup.find('table', {'class': "wikitable sortable"})
ths = table.find_all('th')
cols = [th.text.strip() for th in ths]

toronto_df= pd.DataFrame(columns = ['PostalCode', 'Borough','Neighborhood'])



i = 0
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
        
    toronto_df.loc[i]= [td.text.strip() for td in tds[:4]]
    i = i + 1

    

toronto_df.columns = ['PostalCode','Borough','Neighborhood']
    
print(toronto_df.shape)    
toronto_df.head()


(289, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#####  Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
print(toronto_df.shape)
toronto_df.head(15)

(212, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


##### Replace "Not assigned" Neighbourhoods with Borough

In [6]:
toronto_df.Neighbourhood = np.where(toronto_df.Neighborhood == 'Not assigned', toronto_df.Borough,toronto_df.Neighborhood)
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


##### combined into one row with the neighborhoods separated with a comma

In [7]:
toronto_df = toronto_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
print(toronto_df.shape)
toronto_df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
toronto_df.shape

(103, 3)